In [31]:
from sage.all import *

In [32]:
test_matrix = matrix([[1, 2, 3, 4],
                      [1, 1, 2, 2],
                      [1, 3, 2, 4],
                      [2, 3, 1, 1]])

In [33]:
-vector([1,2,3,4])

(-1, -2, -3, -4)

In [34]:
def q(n):
    return matrix([[n/2 + 1/2, 1],
                   [1, 2]])

In [35]:
q(4)

[5/2   1]
[  1   2]

In [62]:
def Qd(Q = test_matrix, full = False, vectors = False):
    """
    Qd computes and orders d invariants (recording short vectors) of a 
    positive-definite intersection pairing Q.
    
    --- Parameters ---
    
    Optional parameter full : bool
    If True,
    short characteristic covectors are taken in the full hypercube
    -Q_ii <= K_i <= Q_ii (gets full set of short representatives)
    If False or omitted, just look in
    -Q_ii <= K_i<= Q_ii-2 (faster)
    
    Optional paramter vectors : bool
    If True, output a list of short vectors
    with each d invariant         
    """
    # Global variable
    pointer = 0
    
    
    # List of local variables used in the function
    #
    # p, rank(Q), Qinv, K, sqK, dK, QinvK, dlist, QinvKlist, topadjust, Klistno, pointer, i, tempcount,
    # newclassflag, nextK, m, Korder, spinfound, spinindex, S, Uq, biggerthanone, exponents, numberexp, darray, thiselementindex, indexvector, j

    #############################################################################    
    # nextK moves K to the next characteristic vector in the 'hypercuboid'
    def nextK():
        if K[pointer] < m[pointer]-2+topadjust:
            K[pointer] = K[pointer]+2 
        else:
            K[pointer] = -m[pointer] 
            pointer = pointer+1
            if pointer < rank(Q):
                nextK()
            else:
                K[rank(Q)-1] = m[rank(Q)-1]+topadjust
        return
    #############################################################################

    p = Q.determinant()
    Qinv = Q.inverse()
    dlist = [0 for i in range(0, p)]
    QinvKlist = [0 for i in range(0, p)]
    m = vector([Q[i,i] for i in range(0, rank(Q))])
    K = -m

    if full:
        topadjust = 2
    else:
        topadjust = 0
    
    Klistno = 0
    spinfound = False
    
    while K[rank(Q)-1] <= m[rank(Q)-1]-2+topadjust:
        pointer = 0

        QinvK = (1/2)*Qinv*K
        dK = (2*K*QinvK - rank(Q))/4
        QinvK = [QinvK[i] for i in range(0, rank(Q))]

        newclassflag = True

        for i in range(0, Klistno):
            while newclassflag:
                # TODO Find out what this line is doing
                # if nops(remove(is,op(2,op(1,QinvKlist[i]))-QinvK,integer)) == 0:
                #
                # Maple documentation for function ops:
                # https://www.maplesoft.com/support/help/maple/view.aspx?path=op
                if len([x for x in (QinvKlist[i][0][1] - QinvK) if not x.is_integer()]) == 0:
                    newclassflag = False

        if newclassflag:
            Klistno = Klistno + 1 
            dlist[Klistno] = dK
            if not spinfound:
                # if 2* the elements in the list are all integers:
                if len([p for p in map(lambda x : 2*x, QinvK) if not p.is_integer()]) == 0:
                    spinfound = True
                    spinindex = Klistno
            QinvKlist[Klistno] = [[vector(K), QinvK]];
        else: # if newclassflag:
            if dK == dlist[i-1]:
                QinvKlist[i-1] = QinvKlist[i-1].extend([vector(K), QinvK])
            if dK < dlist[i-1]:
                dlist[i-1] = dK
                QinvKlist[i-1] = [[vector(K), QinvK]]

        nextK() # END while K[rank(Q)] <= m[rank(Q)]-2+topadjust:
    
    S, Uq, _ = Q.smith_form() # smith_form() returns a list with three elements, the first being the diagonal form
    biggerthanone = True
    exponents = []

    # the -1 in range(rank(Q), 0, -1) makes the index i count downwards
    for i in range(rank(Q), 0, -1):
        while biggerthanone:
            if S[i,i] == 1:
                biggerthanone = False
            else:
                exponents = range(0, S[i,i]) + exponents

    numberexp = len(exponents)
    darray = exponents
    
    for i in range(1, p):
        thiselementindex = []
        # to concatenate the vectors v1 and v2 use
        # vector( list(v1) + list(v2) )
        right_vector = vector(list([1,1]) + list(QinvKlist[i])) - vector(list([1,1]) + list(QinvKlist[spinindex]))
        indexvector = (1/2)*Uq*right_vector
        for j in range(rank(Q) - numberexp + 1, rank(Q)):
            thiselementindex = thiselementindex.append(indexvector[j] % S[j,j])
            if vectors:
                darray[thiselementindex] = dlist[i].extend([[l, 1].append(QinvKlist[i]) for l in range(0, len(QinvKlist[i]))])
            else:
                darray[op(thiselementindex)] = dlist[i]
    
    return darray

In [63]:
# Assume the given knot has untwisting number +1
# (can be unknotted with a single left-handed nullhomologous twist).
# Then Corollary 4.6 of “Untwisting information” applies to K. (https://arxiv.org/abs/1604.03033)

# Compute the negative definite Goeritz matrix G of K and input it into SageMath
G_10_68 = matrix([[-4,  1,  1,  0,  0,  1,  0],
                  [ 1, -3,  0,  0,  1,  0,  0],
                  [ 1,  0, -2,  1,  0,  0,  0],
                  [ 0,  0,  1, -2,  1,  0,  0],
                  [ 0,  1,  0,  1, -3,  0,  1],
                  [ 1,  0,  0,  0,  0, -2,  1],
                  [ 0,  0,  0,  0,  1,  1, -2]])
# G_10_68 is negative definite, and has determinant -57
G_10_68.determinant()


# The values of mG mod 2 should be
# 0 98/57 50/57 28/19 86/57 56/57 36/19 14/57 2/57
# 24/19 110/57 2/57 30/19 32/57 56/57 16/19 8/57 50/57
# 20/19 2/3 98/57 4/19 8/57 86/57 6/19 32/57 14/57
# 26/19 110/57 110/57 26/19 14/57 32/57 6/19 86/57 8/57
# 4/19 98/57 2/3 20/19 50/57 8/57 16/19 56/57 32/57
# 30/19 2/57 110/57 24/19 2/57 14/57 36/19 56/57 86/57
# 28/19 50/57 98/57.

-57

In [64]:
# Execute the cells with the definition of the function Qd and its dependencies
# and to define the family of matrices q(n) = Q_n 
# representing the intersection pairing on the 4-manifold with boundary
# ±the double branched cover of K (guaranteed by Corollary 4.6).
G = G_10_68

# Compute the list of d-invariants of the double branched cover of K from the Goeritz matrix.

dg = Qd(-G, True, False)
m_G = [dg[i] for i in range(0, determinant(G))] # list the m_G from the previous step.
dq = Qd(q(Determinant(G)), True, False) # list the m_Q.

# If the lists of the m_G and m_Q are not equal modulo 2 (up to reordering),
# then we have successfully obstructed tu(K) = +1.
# If they are equal, we can still try to obstruct
# the existence of isomorphisms using something like the attached Sage worksheet.

# To obstruct tu(K) = -1,
# compute the Goeritz matrix G’ of mirror(K) and repeat the above steps.

UnboundLocalError: local variable 'pointer' referenced before assignment